<a href="https://colab.research.google.com/github/salwazpw/2023_SPK/blob/main/UAS_SPK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np

def topsis(data, criteria_weights, criteria_types):
    print("Data:\n", data.to_string(index=False))
    print("\n")

    # Menghitung matriks terbobot
    weighted_matrix = calculate_weighted_matrix(normalized_matrix, criteria_weights, criteria_types)
    print("Matriks Terbobot:\n", weighted_matrix.to_string(index=False))
    print("\n")

    # Menghitung solusi ideal positif (S+) dan solusi ideal negatif (S-)
    ideal_positive, ideal_negative = calculate_ideal_solutions(weighted_matrix, criteria_types)
    print("Solusi Ideal Positif (S+):\n", ideal_positive)
    print("\n")
    print("Solusi Ideal Negatif (S-):\n", ideal_negative)
    print("\n")

    # Menghitung jarak solusi alternatif terhadap solusi ideal positif dan solusi ideal negatif
    positive_distances = euclidean_distance(weighted_matrix, ideal_positive)
    negative_distances = euclidean_distance(weighted_matrix, ideal_negative)
    print("Jarak Solusi Alternatif ke Solusi Ideal Positif:\n", positive_distances.to_string(index=False))
    print("\n")
    print("Jarak Solusi Alternatif ke Solusi Ideal Negatif:\n", negative_distances.to_string(index=False))
    print("\n")

    # Menghitung nilai preferensi relatif (Ci)
    preference_values = negative_distances / (positive_distances + negative_distances)
    print("Nilai Preferensi Relatif (Ci):\n", preference_values.to_string(index=False))
    print("\n")

    # Menambahkan kolom nilai preferensi relatif ke dalam dataframe
    data['Preference Value'] = preference_values

    # Menambahkan kolom peringkat
    data['Ranking'] = data['Preference Value'].rank(ascending=False).astype(int)

    # Mengurutkan berdasarkan nilai preferensi relatif secara menurun
    sorted_data = data.sort_values(by='Ranking')

    # Mengurutkan berdasarkan nilai preferensi relatif secara menurun
    print("Hasil Akhir:\n", sorted_data.to_string(index=False))
    print("\n")

    return sorted_data

def normalize_matrix(matrix):
    # Menghitung kuadrat dari masing-masing elemen dalam matriks
    squared_matrix = matrix.apply(lambda x: x**2)

    # Menghitung jumlah kuadrat setiap kolom
    column_sums = np.sqrt(squared_matrix.sum())

    # Membagi setiap elemen dalam matriks dengan jumlah kuadrat kolom
    normalized_matrix = matrix / column_sums

    return normalized_matrix

def calculate_weighted_matrix(matrix, criteria_weights, criteria_types):
    # Mengalikan matriks ternormalisasi dengan bobot
    weighted_matrix = matrix.copy()
    for i, col in enumerate(matrix.columns):
        weighted_matrix[col] = matrix[col] * criteria_weights[i]

    return weighted_matrix

def calculate_ideal_solutions(matrix, criteria_types):
    # Mengevaluasi solusi ideal positif dan solusi ideal negatif berdasarkan sifat kriteria
    ideal_positive = matrix.max()
    ideal_negative = matrix.min()

    for i, col in enumerate(matrix.columns):
        if criteria_types[i] == 'Cost':
            ideal_positive[col] = matrix[col].min()
            ideal_negative[col] = matrix[col].max()

    return ideal_positive, ideal_negative

def euclidean_distance(matrix, vector):
    # Menghitung jarak Euclidean antara setiap baris matriks dan vektor
    distances = np.sqrt(np.sum((matrix - vector)**2, axis=1))

    return distances

# Penggunaan fungsi topsis dengan file data.csv, bobot kriteria, dan jenis kriteria
criteria_weights = [0.15, 0.15, 0.1, 0.1, 0.05, 0.1, 0.1, 0.05, 0.1, 0.1]
criteria_types = ['Benefit', 'Benefit', 'Benefit', 'Benefit', 'Benefit', 'Cost', 'Cost', 'Benefit', 'Benefit', 'Benefit']

# Meminta input pengguna untuk memilih sumber data
data_source = input("Pilih sumber data (manual/csv): ")

if data_source == "csv":
    # Meminta input pengguna untuk nama file CSV
    filename = input("Masukkan nama file CSV: ")

    # Membaca data dari file CSV
    data = pd.read_csv(filename)

    # Menghilangkan kolom Id
    data = data.drop('id', axis=1)
    print("Data Setelah Menghilangkan Kolom id:\n", data.to_string(index=False))
    print("\n")

    # Normalisasi matriks
    normalized_matrix = normalize_matrix(data.iloc[:, 1:])
    print("Matriks Normalisasi:\n", normalized_matrix.to_string(index=False))
    print("\n")
else:
    # Meminta input pengguna untuk setiap nilai dalam matriks
    data = pd.DataFrame(columns=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10'])

    num_rows = int(input("Masukkan jumlah Data Alternatif: "))

    for i in range(num_rows):
        row_data = []
        for col in data.columns:
            if i == 0:
                value = float(input(f"Masukkan nilai {col} untuk alternatif 1: "))
            else:
                value = float(input(f"Masukkan nilai {col} untuk alternatif {i+1}: "))
            row_data.append(value)
        data = data.append(pd.Series(row_data, index=data.columns), ignore_index=True)

    # Normalisasi matriks
    normalized_matrix = normalize_matrix(data.iloc[:, :])
    print("Matriks Normalisasi:\n", normalized_matrix.to_string(index=False))
    print("\n")

hasil_topsis = topsis(data, criteria_weights, criteria_types)

Pilih sumber data (manual/csv): csv
Masukkan nama file CSV: Data.csv
Data Setelah Menghilangkan Kolom id:
 Alternatif  C1  C2  C3  C4  C5  C6  C7  C8  C9  C10
        A1   4   4   4   4   4   5   4   4   4    5
        A2   4   4   5   4   5   5   4   4   5    5
        A3   4   5   4   4   4   4   4   4   5    5
        A4   4   3   4   4   3   5   4   4   4    4
        A5   4   4   4   4   5   4   3   4   4    3
        A6   5   4   4   5   5   4   3   4   5    3
        A7   4   5   4   4   4   4   4   5   3    3
        A8   5   4   4   4   4   3   4   4   3    3
        A9   4   4   4   4   5   3   5   5   3    4
       A10   5   4   4   3   4   5   4   3   5    5
       A11   4   4   4   5   4   5   4   2   4    3
       A12   5   4   4   4   5   4   4   2   4    3
       A13   5   4   4   4   4   3   3   4   4    4
       A14   4   4   4   4   4   3   3   5   4    5
       A15   4   4   4   4   4   5   5   5   5    5
       A16   5   5   5   4   4   4   5   4   3    3
       A1